Implementación y explicación del método para el preprocesado del texto.

Implementa una función llamada preprocess_post(text: str) en el archivo core.py.
La función debe limpiar y normalizar los textos de los posts (columna "post") y guardar el resultado en una nueva columna "clean_post".
Describe detalladamente los pasos de preprocesamiento en un notebook de Python (implementacion_modulo_1.ipynb), como eliminación de signos de puntuación, conversión a minúsculas, etc.

---**Importamos el dataframe**--- 
Utilizado posteriormente para columna 'clean_post' y creación del dataframe para losw siguientes módulos.

In [3]:
%pip install numpy=="2.0.2"

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

reddit_df = pd.read_csv('reddit_database_sentiment/reddit_database_sentiment.csv', delimiter=';', quotechar='"', encoding='utf-8', low_memory=False)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\MAM\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\MAM\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\MAM\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\MAM\anaconda3\Lib\site-packages\tor

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

**Análisis de la información de la columna post**

In [33]:
reddit_df['post'][1]

'I\'m cross posting this from /r/cyberlaw, hopefully you guys find it as interesting as I did(it deals with Google Analytics):\n\nSo quite awhile ago, I ordered a Papa John\'s pizza online. My job largely involves looking at ads that appear online, so afterwards I was quick to notice *I was getting a LOT* of Papa Johns ads (especially at night) being served through a Google owned company (DoubleClick media). Yesterday one of these ads popped up again on Youtube (a place that typically serves using the adwords program, not doubleclick), so I decided to copy the URL. \n\nFor those not in the advertising field: Making full use of Google\'s analytics tool means that certain information about the advertising campaign is leaked in the URL.\n\nSo let\'s break it apart: \n\n&gt;http://ad.doubleclick.net/click;h=(junk here);~sscs=?http://googleads.g.doubleclick.net/aclk?sa=l&amp;ai=(junk here)&amp;adurl=http://www.papajohns.com/index.shtm?utm_source=googlenetwork&amp;utm_medium=DisplayCPC&amp;u

Aquí se observa que es necesario:
- Cambiar a minúsculas.
- Quitar saltos de líneas y demás letras escapadas ('\n', '\r', '\t', '\b', '\f').
- Eliminar URLs, que no aportan al análisis de sentimiento.
- Eliminar espacios extra.
- Eliminar caracteres especiales y números.
- Decodificar el html.

In [34]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**Descargas necesarias (descomentar si no se tienen)**


In [35]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

**Explicación paso a paso**

Conversión a minúsculas: \
 Al convertir todo el texto a minúsculas, evitamos duplicados en diferentes casos (por ejemplo, "Data" y "data" se tratan como la misma palabra).
 ```python
    text = text.lower()	

Eliminación de URL's: \
Las URLs generalmente no aportan información relevante para el análisis de contenido y pueden distraer al modelo.
  ```python
    text = re.sub(r'http\S+|https\S+|www\S+', '', text, flags=re.MULTILINE)

Eliminación de menciones de usuario y subreddits: \
Las menciones específicas como @usuario o /r/subreddit son específicas y pueden no ser útiles para la clasificación general.
```python 
text = re.sub(r'@\w+|\/r\/\w+', '', text)

Eliminación de caracteres especiales y números: \
```python
text = text.translate(str.maketrans('', '', string.punctuation))


Eliminación de Números: \
Los números no pueden ser relevantes para el análisis de sentimiento. (Excluyendo en casos de notas o calificaciones)
```python
text = re.sub(r'\d+', '', text)

Tokenización, stopwords: \
Dividir el texto en tokens (palabras) facilita el procesamiento posterior, las stopwords son marcas gramaticales que no aportan al análisis de sentimiento.
```python
tokens = text.split()
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]

En primera instancia, se buscaba lematizar el texto para quedarnos solo con la raíz de las palabras, aunque finalmente se decidió no hacerlo para no perder información relevante.


**Código completo de la función**

In [36]:
def preprocess_post(text: str) -> str:
    """
    Limpia y normaliza el texto de un post de Reddit.

    Parámetros:
    text (str): Texto original del post.

    Retorna:
    str: Texto normalizado para análisis.
    """

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    text = re.sub(r'@\w+|\/r\/\w+', '', text)
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    text = re.sub(r'\d+', '', text)
    
    text = text.lower()

    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    clean_text = ' '.join(tokens)
    
    return clean_text

**Prueba función** \
Se prueba la función con un texto de ejemplo para verificar que el preprocesamiento se realizó correctamente.

In [37]:
test_text = """
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning
"""
clean_text = preprocess_post(test_text)
print("Texto original: ", test_text)
print("Texto limpio: ", clean_text)


Texto original:  
Check out this amazing project on machine learning! Visit https://github.com/user/repo for more details.
Thanks @datascientist for the insights. Also, shoutout to /r/MachineLearning for the support.
Contact me at 123-456-7890. #datascience #machinelearning

Texto limpio:  Check amazing project machine learning Visit detail Thanks insight Also shoutout support Contact datascience machinelearning


**Creación de la columna clean_post y guardado del dataframe**
Ahora trabajaremos para adecuar el dataframe su totalidad, creando la columna clean_post y guardando el dataframe en un nuevo archivo csv.
```python

**Preparación del dataset**
1. Normalizamos las fechas del dataset en caso de que sean necesarias en posteriores módulo, mejorando la calidad de la información.
```python
reddit_df['created_date'] = pd.to_datetime(reddit_df['created_date'], format='%Y-%m-%d %H:%M', errors='coerce')
reddit_df['author_created_date'] = pd.to_datetime(reddit_df['author_created_utc'], unit='s', errors='coerce')


2. Llenamos los valores nulos con un string vacío para evitar problemas en el preprocesamiento.
```python
reddit_df['post'] = reddit_df['post'].fillna('')


3. Aplicamos la función preprocess_post a la columna post.
```python
reddit_df['clean_post'] = reddit_df['post'].apply(preprocess_post)

In [38]:
reddit_df['created_date'] = pd.to_datetime(reddit_df['created_date'], format='%Y-%m-%d %H:%M', errors='coerce')
reddit_df['author_created_date'] = pd.to_datetime(reddit_df['author_created_utc'], unit='s', errors='coerce')

In [39]:
reddit_df['post'] = reddit_df['post'].fillna('')

reddit_df = reddit_df.drop_duplicates()
reddit_df = reddit_df.dropna()

reddit_df['clean_post'] = reddit_df['post'].apply(preprocess_post)


In [40]:
processed_dataset_path = 'processed_dataset.csv'
reddit_df.to_csv(processed_dataset_path, index=False, sep=';')

In [41]:
import zipfile

with zipfile.ZipFile('processed_dataset.csv.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(processed_dataset_path)